<h1><font color="#113D68" size=6>Deep Learning con Python y Keras</font></h1>

<h1><font color="#113D68" size=5>Parte 3. Multilayer Perceptron</font></h1>

<h1><font color="#113D68" size=4>7. Proyecto de regresión</font></h1>

<br><br>
<div style="text-align: right">
<font color="#113D68" size=3>Manuel Castillo Cara</font><br>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>

* [0. Contexto](#section0)
* [1. Topología de linea base](#section1)
* [2. Optimizar el rendimiento con procesamiento de datos](#section2)
* [3. Ajuste de la toplogía](#section3)
    * [3.1. Evaluar una topología más profunda](#section3.1)
    * [3.2. Evaluar una topología más grande](#section3.2)

---
<a id="section0"></a>
# <font color="#004D7F" size=6> 0. Contexto</font>

En este tutorial del proyecto, trabajaremos cómo desarrollar y evaluar modelos de redes neuronales para un problema de regresión:
* Cómo crear un modelo de red neuronal para un problema de regresión.
* Cómo utilizar scikit-learn con Keras para evaluar modelos mediante validación cruzada.
* Cómo realizar la preparación de datos para mejorar la habilidad.
* Cómo Optimizar la topología de red de modelos (tanto en el número de neuronas como de capas).

In [1]:
import tensorflow as tf
# Eliminar warning
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section1"></a>
# <font color="#004D7F" size=6>1. Topología de linea base</font>

En esta sección crearemos un modelo de red neuronal de referencia para el problema de regresión. Comencemos importando todas las funciones y objetos que necesitaremos para este tutorial.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Más información sobre el dataset [Boston House Price](http://lib.stat.cmu.edu/datasets/boston)

In [2]:
# Regresion con el Boston House Price Dataset
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

# Cargamos el set de datos
dataframe = pd.read_csv("Datasets/housing.csv", delim_whitespace=True, header=None) # delim_whitespace para indicar que el csv separa columnas por espacios
dataset = dataframe.values

# Separamos entre variables de entrada (X) y salida (Y)
X = dataset[:,0:13] # si me lo importa como objeto debo hacer la conversión a float, como en el problema de clasificación
Y = dataset[:,13]

Comencemos por definir la función que crea nuestro modelo de línea de base. 
1. Tiene una sola capa oculta completamente conectada con el mismo número de neuronas que los atributos de entrada (13). 
2. La red utiliza la función de activación ReLu para la capa oculta. 
    * Al ser problema de regresión no tiene función de activación la capa de salida
3. Utiliza el algoritmo de optimización ADAM y se optimiza una función de pérdida de error cuadrático medio (comparable en terminos del problema). 
4. El objeto Wrapper para regresión se llama `KerasRegressor`. 
5. Evaluamos este modelo de línea de base con 10-fold.

In [8]:
# Creamos un modelo simple con una capa oculta con 13 neuronas (completamente conectada con la capa de entrada)
# Usamos función de activación ReLu, pero en problemas de Regresión no se utiliza función de activación en la capa de salida

# Defino modelo base en la función
def baseline_model():
    # creo el modelo
    model = Sequential()
    model.add(Dense(13,input_dim=13,activation='relu')) # capa oculta de 13 neuronas de salida (igual que de entrada) con función de activación ReLu
    model.add(Dense(1)) # capa de salida con un solo valor de salida, que no necesita función de activación (Regresión)
    # si estandarizo Y entre los valores 0 y 1 puedo usar una función de activación sigmoidal para predecir en la capa de salida
    # compilo el modelo
    model.compile(loss='mean_squared_error',optimizer='adam') # usamos el MSE como métrica
    return model
    
# evaluo el modelo
estimator = KerasRegressor(model=baseline_model, epochs=100, batch_size=5, verbose=0) # Objeto estimador
kfold = KFold(n_splits=10) # Objeto de entrenamiento, podemos agg el shuffle si queremos
results = cross_val_score(estimator,X,Y,cv=kfold) # tener X y Y en formato numérico
print("MSE Modelo Base: %.2f (%.2f)" % (results.mean(),results.std()))
# Me devuelve el MSE con signo negativo (ignorar signo)
# Vemos que el modelo base me arroja buenos resultados (el error esta expresado en miles de dólares), sin embargo vamos a intentar optimizar su rendimiento

C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs

MSE Linea Base: -0.17 (0.72)


---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section2"></a>
# <font color="#004D7F" size=6>2. Optimizar el rendimiento con procesamiento de datos</font>

Podemos utilizar el marco de trabajo [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) para realizar la **estandarización** durante el proceso de evaluación del modelo, dentro de cada fold. 

In [12]:
# Todos los artibutos de entrada varían mucho en sus escalas, por lo que es buena practica hacer un escalamiento de los mismos antes de procesar (estandarización)
# Librerias
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline # para hacerlo dentro de cada fold y no aprenderlo de folds anteriores lo hacemos en Pipeline

# Evaluo el modelo con el set estandarizado
estimators = []
estimators.append(('standarize', StandardScaler)) # estandarizo mis datos
estimators.append(('NN(red neuronal)', KerasRegressor(model=baseline_model, epochs=100, batch_size=5, verbose=0)))
kfold = KFold(n_splits=10) # Objeto de entrenamiento, podemos agg el shuffle si queremos
results = cross_val_score(estimator,X,Y,cv=kfold) # tener X y Y en formato numérico
print("MSE Modelo Estandarizado: %.2f (%.2f)" % (results.mean(),results.std()))
# Vemos que el resultado del modelo mejoró un poco

C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs

MSE Modelo Estandarizado: -0.76 (1.75)


<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
**Trabajo a realizar:** Una extensión adicional de esta sección sería aplicar de manera similar un cambio de escala a la variable de salida, como normalizarla al rango de 0 a 1 y usar una función de activación Sigmoide o similar en la capa de salida para reducir las predicciones de salida al mismo rango.

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section3"></a>
# <font color="#004D7F" size=6>3. Ajuste de la topología</font>

En esta sección evaluaremos dos topologías (más profunda y amplia) para mejorar aún más el rendimiento del modelo.

<a id="section3.1"></a>
# <font color="#004D7F" size=5>3.1. Evaluar una topología más profunda</font>

Una forma de mejorar el rendimiento de una red neuronal es agregar más capas para hacerla más profunda. En este caso con aproximadamente la mitad del número de neuronas. Nuestra topología de red ahora se ve así:
```
    13 entradas -> [13 -> 6] -> 1 salida
```
Podemos evaluar esta topología de red de la misma manera que anteriormente, al mismo tiempo que usamos la estandarización que demostró un mejor rendimiento.

In [13]:
# Probamos agregando una segunda capa oculta con 6 neuronas al modelo (casi siempre la tendencia es ir reduciendo a la mitad las neuronas de las sucesivas capas)

# Defino el modelo base (ahora más profundo/largo)
def larger_model():
   # creo el modelo
    model = Sequential()
    model.add(Dense(13,input_dim=13,activation='relu')) 
    model.add(Dense(6,activation='relu')) # en esta nueva capa extrae lo más importante de la primera
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',optimizer='adam') 
    return model
    
# Evaluo el modelo con el set estandarizado
estimators = []
estimators.append(('standarize', StandardScaler)) # estandarizo mis datos
estimators.append(('Deep-NN', KerasRegressor(model=larger_model, epochs=100, batch_size=5, verbose=0)))
kfold = KFold(n_splits=10) # Objeto de entrenamiento, podemos agg el shuffle si queremos
results = cross_val_score(estimator,X,Y,cv=kfold) # tener X y Y en formato numérico
print("MSE Deep NN: %.2f (%.2f)" % (results.mean(),results.std())) # vemos los ressultados de esta nueva red neuronal profunda

# Vemos que el rendimiento del modelo mejora, por lo quenos es conveniente añadir esta nueva capa oculta

C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs

MSE Deep NN: -0.38 (0.89)


<a id="section3.2"></a>
# <font color="#004D7F" size=5>3.2. Evaluar una topología más grande</font>

Otro enfoque para aumentar la capacidad de representación del modelo es crear una red más amplia. Aquí, hemos aumentado el número de neuronas en la capa oculta en de 13 a 20. La topología de nuestra red más amplia se puede resumir de la siguiente manera:

```
    13 entradas -> [20] -> 1 salida
```

In [14]:
# Ahora volvemos a la capa oculta única, pero en lugar de 13 neuronas probamos con 20

# Defino el modelo base (ahora más grande/ancho)
def wider_model():
    # creo el modelo
    model = Sequential()
    model.add(Dense(20,input_dim=13,activation='relu')) # ahora de 20 neuronas
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',optimizer='adam') 
    return model
    
# Evaluo el modelo con el set estandarizado
estimators = []
estimators.append(('standarize', StandardScaler)) # estandarizo mis datos
estimators.append(('Wider-NN', KerasRegressor(model=wider_model, epochs=100, batch_size=5, verbose=0))) # probar menos epocas, ya que el conjunto de datos no es tan grande
kfold = KFold(n_splits=10) # Objeto de entrenamiento, podemos agg el shuffle si queremos
results = cross_val_score(estimator,X,Y,cv=kfold) # tener X y Y en formato numérico
print("MSE Wider NN: %.2f (%.2f)" % (results.mean(),results.std())) # vemos los ressultados de esta nueva red neuronal ancha

# Parece que con esta configuración el modelo mejora en terminos de predicción, pero no de varianza con respecto al modelo del bloque anterior
# Podemos seguir probando variantes de modelos

C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs

MSE Wider NN: -0.35 (0.93)


La construcción del modelo ve una caída adicional en el error a aproximadamente 22 mil dólares cuadrados. Este no es un mal resultado para este problema.

<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>